In [0]:
import sys
import pandas
import numpy as np
import random as rn
import os
import tensorflow as tf
from keras import backend as K
# Set up random seed to
# get the same results every
# time you train your model.
rs=5

np.random.seed(rs)
rn.seed(rs)
tf.set_random_seed(rs)
K.set_session(sess)

In [0]:
data=pandas.read_csv("data/airline_data.csv", header=2)

In [0]:
data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84700.0,97400.0,104900.0,96600.0,96100.0,99000.0,101700.0,111200.0,100400.0,93000.0,76200.0,129100.0,181000.0,200200.0,220400.0,225500.0,184400.0,217900.0,201000.0,220000.0,241400.0,212300.0,212300.0,197000.0,238400.0,250400.0,255600.0,89600.0,52700.0,140200.0,149705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999127.0,2.279341e+06,1.737962e+06,2044188.0,2.209428e+06,1929907.0,1917924.0,1858558.0,NaN
2,Angola,AGO,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155400.0,282700.0,308000.0,546100.0,635200.0,801100.0,890500.0,952500.0,689600.0,894200.0,727000.0,746000.0,750000.0,509800.0,451500.0,456000.0,440000.0,333500.0,519000.0,552500.0,585000.0,555000.0,552900.0,293800.0,235448.0,193317.0,189619.0,198253.0,222032.0,239795.0,262666.0,277361.0,283887.0,274869.0,1010194.0,9.877980e+05,1.132424e+06,1321872.0,1.409952e+06,1244491.0,1482546.0,1581201.0,NaN
3,Albania,ALB,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9100.0,13000.0,29000.0,55400.0,21200.0,91500.0,136622.0,146273.0,137569.0,158888.0,179543.0,195702.0,213315.0,238913.0,243691.0,231263.0,768533.0,8.297789e+05,8.143397e+05,865848.0,1.516320e+05,NaN,26634.0,103151.0,NaN
4,Andorra,AND,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Kosovo,XKX,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yemen, Rep.",YEM,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25900.0,54500.0,57000.0,61500.0,66500.0,70000.0,262800.0,275000.0,295000.0,295000.0,310000.0,310000.0,352100.0,465900.0,455800.0,426200.0,589700.0,575500.0,635500.0,667900.0,671100.0,540500.0,800400.0,847900.0,790600.0,375000.0,587500.0,706900.0,765000.0,731000.0,841609.0,841366.0,869296.0,844284.0,1022079.0,1083404.0,1162316.0,1073493.0,1064878.0,1050486.0,1536556.0,1.492073e+06,1.299707e+06,1664220.0,1.665552e+06,443685.0,242723.0,132571.0,NaN
261,South Africa,ZAF,"Air transport, passengers carried",IS.AIR.PSGR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1519000.0,1659500.0,1868100.0,2313800.0,2763600.0,2936400.0,2998800.0,3076800.0,3211200.0,3783700.0,4116000.0,4159800.0,4091200.0,3951800.0,4461400.0,4337300.0,3825000.0,4386900.0,5125900.0,5641500.0,5364900.0,4818600.0,4685000.0,5581500.0,5801600.0,6395500.0,7183100.0,7274200.0,6479500.0,7403800.0,8000757.0,7948374.0,8052660.0,9159815.0,9878598.0,11844657.0,12932671.0,12870

In [0]:
def get_raw_xy(data, column='Country Name', value='World'):
    v=data.loc[data[column] == value].iloc[:,17:62]
    x=[ v for v in v.values.astype('float32')[0] ]
    return x, [ int(vv) for vv in v.columns ]

In [0]:
def get_vpo(values, goback = 1):
    yy=[np.nan]*len(values)
    for i, v in enumerate(values):
        if i+goback>len(values)-1:
            break
        yy[i+goback] = [values[f] for f in range(i, i+goback)]
        
    return yy

In [0]:
def get_data(f='data/airline_data.csv', logme=lambda x:x):
    # Reading our data, skipping 2 lines of comments.
    d=pandas.read_csv(f, header=2)
    values, years = get_raw_xy(d)
    logme('Raw values')
    rv=list(zip(years, values))
    logme(rv)
    # To base prediction on more that year's value in the past
    # replace get_vpo with get_vpo2
    # and set goback parameter as the same as inputs in an entry
    # entry in confs inside train.py
    past_values=get_vpo(values)
    logme("Previous year's values X/ this year's value Y")
    logme("For every value from X expect value from Y")
    dpo=list(zip([1972]+years, past_values, years, values))
    logme(dpo)
    return years[1:], past_values[1:], values[1:]

In [0]:
from pprint import pprint
get_data(logme=pprint)

'Raw values'
[(1973, 401571800.0),
 (1974, 421145200.0),
 (1975, 432276500.0),
 (1976, 471773400.0),
 (1977, 513269280.0),
 (1978, 576090000.0),
 (1979, 648400600.0),
 (1980, 641872900.0),
 (1981, 640619400.0),
 (1982, 654482100.0),
 (1983, 685101600.0),
 (1984, 732410300.0),
 (1985, 783198100.0),
 (1986, 842594300.0),
 (1987, 904838100.0),
 (1988, 953896000.0),
 (1989, 983208800.0),
 (1990, 1024976640.0),
 (1991, 1133228200.0),
 (1992, 1145436700.0),
 (1993, 1142265200.0),
 (1994, 1233233400.0),
 (1995, 1302891600.0),
 (1996, 1390963700.0),
 (1997, 1455104300.0),
 (1998, 1466961800.0),
 (1999, 1562256300.0),
 (2000, 1674064800.0),
 (2001, 1655230200.0),
 (2002, 1627404900.0),
 (2003, 1665309300.0),
 (2004, 1888695300.0),
 (2005, 1969590800.0),
 (2006, 2072414000.0),
 (2007, 2209136400.0),
 (2008, 2208218600.0),
 (2009, 2249728500.0),
 (2010, 2628261400.0),
 (2011, 2786953700.0),
 (2012, 2894055000.0),
 (2013, 3048275200.0),
 (2014, 3227291400.0),
 (2015, 3466478600.0),
 (2016, 3705101

([1974,
  1975,
  1976,
  1977,
  1978,
  1979,
  1980,
  1981,
  1982,
  1983,
  1984,
  1985,
  1986,
  1987,
  1988,
  1989,
  1990,
  1991,
  1992,
  1993,
  1994,
  1995,
  1996,
  1997,
  1998,
  1999,
  2000,
  2001,
  2002,
  2003,
  2004,
  2005,
  2006,
  2007,
  2008,
  2009,
  2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017],
 [[401571800.0],
  [421145200.0],
  [432276500.0],
  [471773400.0],
  [513269280.0],
  [576090000.0],
  [648400600.0],
  [641872900.0],
  [640619400.0],
  [654482100.0],
  [685101600.0],
  [732410300.0],
  [783198100.0],
  [842594300.0],
  [904838100.0],
  [953896000.0],
  [983208800.0],
  [1024976640.0],
  [1133228200.0],
  [1145436700.0],
  [1142265200.0],
  [1233233400.0],
  [1302891600.0],
  [1390963700.0],
  [1455104300.0],
  [1466961800.0],
  [1562256300.0],
  [1674064800.0],
  [1655230200.0],
  [1627404900.0],
  [1665309300.0],
  [1888695300.0],
  [1969590800.0],
  [2072414000.0],
  [2209136400.0],
  [2208218600.0],
  [2249728500.0]

In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense

import math
import os
import sys
import pickle

In [0]:
def get_mlp(inputs=1):
    model = Sequential()
    # input_dim is our input layer
    # A number of neurons here is equal
    # to the number of values we're using for
    # prediction, which is just 1
    #
    # Next is our hidden layer, number of neurons
    # should be between number of neurons in
    # input and output layers, here we have 1 again
    #
    # Activation function is one of key component
    # in the way our network will learn.
    # Rectified linear unit (ReLU) or Rectifier
    # activation function is a great default choice
    # for most problems.
    model.add(Dense(1, input_dim=inputs, activation='relu'))
    # Last layer is our output layer
    # Number of neurons here is the same
    # as number of output values
    # Here we just want to get one prediction,
    # so just one neuron here
    model.add(Dense(1))
    return model

In [0]:
model=get_mlp
inputs=1

In [0]:
def train_test_split(rawx, xpo):
    train_size=int(len(rawx)*0.80)
    test_size=int(len(rawx)*0.20)
    #print(train_size, test_size, len(rawx))
    train_x, train_y = np.array(rawx[:train_size]), np.array(xpo[:train_size])
    test_x, test_y = np.array(rawx[train_size:]), np.array(xpo[train_size:])
    return train_x, train_y, test_x, test_y

In [0]:
def train_model(train_x, train_y, epochs=5, batches=1):
    model=get_mlp
    inputs=1
    model=model(inputs)
    # Loss is our loss or cost function - mean_squared_error
    # is a good choice assuming we don't have a lot of "outliers"
    # in our dataset.
    # Adam optimizer works great for most problems.
    #
    # Metrics are loss metrics that we want to have available for each epoch,
    # so we can review how are we doing at each training stage.
    # mse is mean_squared_error, mpe is mean_absolute_percentage_error
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','mape'])
    # Here we're starting our training
    history=model.fit(train_x, train_y, verbose=2, epochs=epochs, batch_size=batches)
    return model, history

In [0]:
years, past_values, values=get_data()
X, Y = past_values, values
train_x, train_y, test_x, test_y = train_test_split(X, Y)
model, history=train_model(train_x, train_y)
# X[-1:], Y[-1:]

Epoch 1/5
 - 2s - loss: 461563078022573952.0000 - mean_squared_error: 461563078022573952.0000 - mean_absolute_percentage_error: 52.1797
Epoch 2/5
 - 0s - loss: 360383403135823296.0000 - mean_squared_error: 360383403135823296.0000 - mean_absolute_percentage_error: 45.9086
Epoch 3/5
 - 0s - loss: 278578115758784512.0000 - mean_squared_error: 278578115758784512.0000 - mean_absolute_percentage_error: 40.5697
Epoch 4/5
 - 0s - loss: 215586637005410432.0000 - mean_squared_error: 215586637005410432.0000 - mean_absolute_percentage_error: 35.5369
Epoch 5/5
 - 0s - loss: 166288440906983488.0000 - mean_squared_error: 166288440906983488.0000 - mean_absolute_percentage_error: 30.7228


In [0]:
np.array(values[-2:])

array([3.7051018e+09, 3.9788493e+09], dtype=float32)

In [0]:
model.predict(np.array(values[-1:]))

array([[5.366744e+09]], dtype=float32)